# Raman NMF analysis

## Introduction

This script uses NMF decomposition from Scikit-Learn
to produce some informative graphical output on map scans.

**ATTENTION:** Since there is no package yet, you need to put the scripts "read_WDF" and "utilities" in your current working directory.

You should first choose the data file with the map scan in the .wdf format.  
Then set the initialization dictionary values.

_That's it!_
___

- First plot: (repeated several times troughout the script)  Shows all of your spectra (with navigation buttons)
- **Second plot**: Plots each of the components found by NMF
- **Third plot**: The heatmap of the mixing coefficients
> (shows the abundance of each component troughout the map)  
When you double-click on a pixel on this map,
it will pop-up another plot
showing the spectra recorded at this point,
together with the contributions of each component

## Imports

In [1]:
# -*- coding: utf-8 -*-
%matplotlib tk
import os
import numpy as np
import pandas as pd
from sklearn import decomposition
from sklearn.neighbors import LocalOutlierFactor
from scipy.signal import savgol_filter, correlate2d
from scipy.ndimage import median_filter
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib import colors
from matplotlib.widgets import Button
import seaborn as sns
from tkinter import filedialog, Tk, messagebox
from timeit import default_timer as time
from read_WDF import convert_time, read_WDF
from utilities import NavigationButtons, clean, rolling_median, slice_lr, baseline_als
from warnings import warn
#import deconvolution

sns.set()

## Loading Data

In [2]:
# From .WDF files:

data_folder = './Data'
prompt_for_images = 'Choose a .wdf file :'
# -----------------------Choose a file-----------------------------------------
root = Tk()
root.withdraw()

filename, = filedialog.askopenfilenames(initialdir=data_folder,
                                        title=prompt_for_images,
                                       filetypes=[("Wire File",'.wdf')])


# Reading the data from the .wdf file
spectra, sigma, params, map_params, origins =\
                            read_WDF(filename, verbose=True)

Reading the file: "carto - 2h_Copy.wdf"


================================= Block : WDF1 =================================
size: 512, offset: 0
WdfFlag--------------------------------- : 	WdfXYXY
PointsPerSpectrum----------------------- : 	891
Capacity-------------------------------- : 	20
Count----------------------------------- : 	20
AccumulationCount----------------------- : 	1
YlistLength----------------------------- : 	1
XlistLength----------------------------- : 	891
DataOriginCount------------------------- : 	5
ApplicationName------------------------- : 	WiRE
ApplicationVersion---------------------- : 	5.1.0 build 8867
ScanType-------------------------------- : 	Static
MeasurementType------------------------- : 	Map
StartTime------------------------------- : 	Fri Nov 22 16:24:57 2019
EndTime--------------------------------- : 	Fri Nov 22 16:58:25 2019
SpectralUnits--------------------------- : 	Counts
LaserWaveLength------------------------- : 	532.06
Title----------------------


- **"spectra"** is a 2D numpy array containing the intensities
    recorded at each point in a map scan.  
    It is of shape:
    `(N°_measurement_points, N°_RamanShifts)`
- **"sigma"** is a 1D numpy array containing all the ramans shift values  
    Its' length is `N°_RamanShifts`
- **"params"** is a dictionnary containing measurement parameters
- **"map_params"** is dictionnary containing map parameters
- **"origins"** is a pandas dataframe giving detail on each point in the map scan
    (time of measurement, _coordinates and some other info).
    
> _Note: It should be noted that the timestamp
    recorded in the origins dataframe is in the Windows 64bit format,
    if you want to convert it to the human readable format,
    you can use the imported "convert_time" function_


In [38]:
see_all_spectra = NavigationButtons(sigma, spectra, autoscale_y=True)

## Initialization
_The next cell is basically the only cell you need to modify._

 - SliceValues: list or tuple of two floats:  Used to isolate the zone you're interested in
 - NMF_NumberOfComponents: int: Number of components for the NMF decomposition
 - PCA_components: int or float from the interval [0,1]: number of components to use for the PCA smoothing. If the number given is the positive float<=1, then it is regarded as the ratio of the total variance to preserve after applying the PCA simplification. The algorithm will chose the proper number of components so to cover the given variance ratio.
 - The last two parameters may help you isolate the physical zone of interest on your map or slice


In [4]:
initialization = {'SliceValues': [160, 1300],  # Use None to count all
                  'NMF_NumberOfComponents': 6,
                  'PCA_components': 0.998,
                  # Put in the int number from 0 to _n_y:
                  'NumberOfLinesToSkip_Beggining': 0,
                  # Put in the int number from 0 to _n_y - previous element:
                  'NumberOfLinesToSkip_End': 0,
                  'BaselineCorrection': True,
                  'CosmicRayCorrection': False,
                  'AbsoluteScale': False}


## Pretreatement

However, if you were to load the data from some other source (like .txt files), it is highely likely that you will not have accès to all the "metadata" contained in the original recording. Those concern mainly measurement parameters and map parameters. You should than make the extra effort of providing at least the following:

In [6]:
# put the retreived number of measurements in a variable
# with a shorter name, as it will be used quite often:
try:
    _n_points = int(params['Count'])
except (NameError, KeyError):
    _n_points = len(spectra)
try:
    if params['MeasurementType'] == 'Map':
        # Finding in what axes the scan was taken:
        _x_index, _y_index = np.where(map_params['NbSteps'] > 1)[0]
except (NameError, KeyError):
    _x_index, _y_index = 0, 1

try:
    if params['MeasurementType'] == 'Map':
        # ATTENTION : from this point on in the script,
        # the two relevant dimensions  will be called
        # X and Y regardless if one of them is Z in reality (for slices)
        _n_x, _n_y = map_params['NbSteps'][[_x_index, _y_index]]
except (NameError, KeyError):
    while True:
        _n_x = int(input("Enter the total number of measurement points along x-axis: "))
        _n_y = int(input("Enter the total number of measurement points along y-axis: "))
        if _n_x*_n_y == _n_points:
            print("That looks ok.")
            break
        elif _n_x * _n_y != _n_points:
            warn("\nWrong number of points. Try again:")
            continue
        break

try:
    if params['MeasurementType'] == 'Map':    
        _s_x, _s_y = map_params['StepSizes'][[_x_index, _y_index]]
except (NameError, KeyError):
    _s_x = int(input("Enter the size of the step along x-axis: "))
    _s_y = int(input("Enter the size of the step along y-axis: "))
    print("ok")

try:
    if params['MeasurementType'] == 'Map':
        if (initialization['NumberOfLinesToSkip_Beggining']
                + initialization['NumberOfLinesToSkip_End']) > _n_y:
            raise SystemExit('You are skiping more lines than present in the scan.\n'
                             'Please revise your initialization parameters')
        _n_yy = _n_y - initialization['NumberOfLinesToSkip_End'] -\
                       initialization['NumberOfLinesToSkip_Beggining']
    else:
        raise SystemExit("Can't yet handle this type of scan")
except:
    pass

Enter the total number of measurement points along x-axis:  5
Enter the total number of measurement points along y-axis:  4


That looks ok.


Enter the size of the step along x-axis:  1
Enter the size of the step along y-axis:  1


ok


### Slicing

In [7]:
# Isolating the part of the spectra that interests us
try:
    pos_left = initialization["SliceValues"][0]
except (NameError, ValueError, TypeError, KeyError):
    pos_left = None
try:
    pos_right = initialization["SliceValues"][1]
except (NameError, ValueError, TypeError, KeyError):
    pos_right = None
spectra_kept, sigma_kept = slice_lr(spectra, sigma,
                                    pos_left=pos_left,
                                    pos_right=pos_right)

# Removing the lines from top and/or bottom of the map
try:
    skip_lines_up = initialization['NumberOfLinesToSkip_End']
except (NameError, ValueError, KeyError):
    skip_lines_up = 0
_start_pos = skip_lines_up * _n_x

try:
    skip_lines_down = initialization['NumberOfLinesToSkip_End']
except (NameError, ValueError, KeyError):
    skip_lines_down = 0

if skip_lines_down == 0:
    _end_pos = None
else:
    _end_pos = -np.abs(skip_lines_down) * _n_x

spectra_kept = spectra_kept[_start_pos:_end_pos]



# =============================================================================
# ATTENTION: This next line is likely buggy. The columns containing the coor-
# dinates are not necessarily the ones indicated with _x_index ans _y_index!
# =============================================================================
#_coordinates = origins.iloc[_start_pos:_end_pos, [_x_index+1, _y_index+1]]

## Smoothing using pca

In [13]:
# %%
# =============================================================================
#                                     PCA...
# =============================================================================
pca = decomposition.PCA(n_components=initialization['PCA_components'])
pca_fit = pca.fit(corrected_spectra)
#pca_fit.n_components = 0.99#min(initialization['PCA_components'], _n_points, len(sigma_kept))
spectra_reduced = pca_fit.transform(corrected_spectra)
spectra_denoised = pca_fit.inverse_transform(spectra_reduced)

In [39]:
# =============================================================================
#                  showing the smoothed spectra
# =============================================================================

_s = np.stack((corrected_spectra, spectra_denoised), axis=-1)
check_pca = NavigationButtons(sigma_kept, _s, autoscale_y=True,
                                    label=["corrected spectra", "pca denoised"],
                                    #title=[f"variance: {pca.explained_variance_ratio_[i]:.3f}" for i in range(pca.n_components_)],
                                    figsize=(12, 12))

### Finding the baseline using the asynchronous least squares method

In [8]:
if initialization['BaselineCorrection']:
    b_line = baseline_als(spectra_denoised)
else:
    b_line = np.zeros_like(spectra_kept)

# Remove the eventual offsets:
corrected_spectra = spectra_denoised - b_line
corrected_spectra -= np.min(corrected_spectra, axis=1)[:,np.newaxis]

In [9]:
# Visualise the baseline correction:
_baseline_stack = np.stack((spectra_denoised, b_line, corrected_spectra), axis=-1)
check_baseline = NavigationButtons(sigma_kept, _baseline_stack, autoscale_y=True,
                                   label=['original spectra', 'baseline', 'baseline corrected spectra'])

### Finding the Cosmic Rays with nearest neghbour and correcting them with median filter
**Note:** _This is method is quite robust in finding Cosmic Ray candidates. Nevertheless, some more effort shoud be put in finding a better way of correcting thus found spectra. At the moment, the code simply uses the median value of the neigbourhood._


In [10]:
# This method identifies spectra which differ a lot from their neighbours:

clf = LocalOutlierFactor(n_neighbors=5, n_jobs=-1)

prd = clf.fit_predict(corrected_spectra)

CR_cand_ind = np.where(prd==-1)[0]

print(CR_cand_ind)

[]


In [42]:
from sklearn.metrics import pairwise_distances
ttt = pairwise_distances(spectra_denoised)

In [43]:
plt.Figure()
sns.heatmap()

(20, 20)

In [11]:
if len(CR_cand_ind) > 0:
    # Find the median value for each spectra, but only with regard to its' neighbours from the same line
    med_spectra_x = rolling_median(corrected_spectra.reshape(_n_yy, _n_x, len(sigma_kept)), w_size=5, ax=1,
                               mode='mirror').reshape((-1,len(sigma_kept)))

    titles = [f"candidate from Nearest Neighbour\noriginal spectra N°{i} " for i in np.nditer(CR_cand_ind)]
    _ss = np.stack((spectra_kept[CR_cand_ind], corrected_spectra[CR_cand_ind], med_spectra_x[CR_cand_ind]), axis=-1)
    NavigationButtons(sigma_kept, _ss, autoscale_y=True, title=titles,
                      label=['original', 'baseline corrected', 'median correction of CR']);

In [12]:
# Apply the correction
# (just replace the whole spectra containing the cosmic ray with the median spectra of its' neighborhood)
if len(CR_cand_ind) > 0:
    corrected_spectra[CR_cand_ind] = med_spectra_x[CR_cand_ind]

## The NMF step

In [16]:
# =============================================================================
#                                   NMF step
# =============================================================================
spectra_cleaned = clean(sigma_kept, spectra_denoised, mode='area')

_n_components = initialization['NMF_NumberOfComponents']
nmf_model = decomposition.NMF(n_components=_n_components, init='nndsvda', max_iter=7, l1_ratio=1)
_start = time()
print('starting nmf... (be patient, this may take some time...)')
mix = nmf_model.fit_transform(spectra_cleaned)
components = nmf_model.components_
reconstructed_spectra1 = nmf_model.inverse_transform(mix)
_end = time()
print(f'nmf done in {_end - _start:.2f}s')

starting nmf... (be patient, this may take some time...)
nmf done in 0.02s


In [18]:
# =============================================================================
#                    preparing the mixture coefficients
# =============================================================================

mix.resize(int(_n_x*_n_y), _n_components, )
_start_pos = initialization['NumberOfLinesToSkip_Beggining'] * _n_x
mix = np.roll(mix, _start_pos, axis=0)
_comp_area = np.empty(_n_components)
for _z in range(_n_components):
    # area beneath each component:
    _comp_area[_z] = np.trapz(components[_z])#, x=sigma_kept)
    components[_z] /= _comp_area[_z]  # normalizing the components by area
    # renormalizing the mixture coefficients:
    mix[:, _z] *= _comp_area[np.newaxis, _z]
mix /= np.sum(mix, axis=-1)[:,np.newaxis]
spectra_reconstructed = np.dot(mix, components)
_mix_reshaped = mix.reshape(_n_y, _n_x, _n_components)

## Main plots

In [19]:
# %%
# =============================================================================
#                    Plotting the components....
# =============================================================================
sns.set()  # to make plots pretty :)

# to keep always the same colors for the same components:
col_norm = colors.Normalize(vmin=0, vmax=_n_components)
color_set = ScalarMappable(norm=col_norm, cmap="brg")

# infer the number of subplots and their disposition from n_components
fi, _ax = plt.subplots(int(np.floor(np.sqrt(_n_components))),
                       int(np.ceil(_n_components /
                                   np.floor(np.sqrt(_n_components))
                                   )))
if _n_components > 1:
    _ax = _ax.ravel()
else:
    _ax = [_ax]
for _i in range(_n_components):
    _ax[_i].plot(sigma_kept, components[_i].T, color=color_set.to_rgba(_i))
    _ax[_i].set_title(f'Component {_i}')
    _ax[_i].set_yticks([])
try:
    fi.text(0.5, 0.04,
            f"{params['XlistDataType']} recordings"
            f"in {params['XlistDataUnits']} units",
            ha='center')
except:
    pass

In [37]:
# %%
# =============================================================================
#                       Plotting the main plot...(heatmap)
# =============================================================================
_n_fig_rows = int(np.floor(np.sqrt(_n_components)))
_n_fig_cols = int(np.ceil(_n_components / np.floor(np.sqrt(_n_components))))
fig, _ax = plt.subplots(_n_fig_rows, _n_fig_cols,
                        sharex=True, sharey=True)
if _n_components > 1:
    _ax = _ax.ravel()
else:
    _ax = [_ax]


def onclick(event):
    '''Double-clicking on a pixel will pop-up the (cleaned) spectrum
    corresponding to that pixel, as well as its deconvolution on the components
    and again the reconstruction for visual comparison'''
    if event.inaxes:
        x_pos = int(np.floor(event.xdata))
        y_pos = int(np.floor(event.ydata))
        broj = int(y_pos*_n_x + x_pos)
        spec_num = int(y_pos*_n_x - _start_pos + x_pos)

        if event.dblclick:
            ff, aa = plt.subplots()
            aa.scatter(sigma_kept, spectra_cleaned[spec_num], alpha=0.3,
                       label=f'(cleaned) spectrum n°{broj}')
            aa.plot(sigma_kept, spectra_reconstructed[broj], '--k',
                    label='reconstructed spectrum')
            for k in range(_n_components):
                aa.plot(sigma_kept, components[k]*mix[broj][k],
                        color=color_set.to_rgba(k),
                        label=f'Component {k} contribution'
                              f'({mix[broj][k]*100:.1f}%)')

# This next part is to reorganize the order of labels,
# so to put the scatter plot first
            handles, labels = aa.get_legend_handles_labels()
            order = list(np.arange(_n_components+2))
            new_order = [order[-1]]+order[:-1]
            aa.legend([handles[idx] for idx in new_order],
                      [labels[idx] for idx in new_order])
            aa.set_title(f'deconvolution of the spectrum from: '
                         f'line {y_pos} & column {x_pos}')
            ff.show()
    else:
        print("you clicked outside the canvas, you bastard :)")


_xcolumn_name = ['X', 'Y', 'Z'][_x_index]
_ycolumn_name = ['X', 'Y', 'Z'][_y_index]

#################################################################################
############## This formatting should be adapted case by case ###################
try:
    _y_ticks = [str(int(x/1000))+'mm' for x in
                np.asarray(origins[_ycolumn_name].iloc[:_n_x*_n_y:_n_x])]
    _x_ticks = [str(int(x/1000))+'mm' for x in
                np.asarray(origins[_xcolumn_name].iloc[:_n_x])]
except:
    pass
#################################################################################
if initialization['AbsoluteScale'] == True:
    scaling = {'vmin': 0, 'vmax': 1}
else:
    scaling={}

for _i in range(_n_components):
    sns.heatmap(_mix_reshaped[:, :, _i], ax=_ax[_i], cmap="jet", annot=False, **scaling)
#    _ax[_i].set_aspect(_s_y/_s_x)
    _ax[_i].set_title(f'Component {_i}', color=color_set.to_rgba(_i),
                      fontweight='extra bold')
fig.suptitle('Heatmaps showing the abundance of individual components'
             ' throughout the scanned area.')
fig.canvas.mpl_connect('button_press_event', onclick)

6

## Appendix - save results

In [31]:
# =============================================================================
#        saving some data for usage in other software (Origin, Excel..)
# =============================================================================
_basic_mix = pd.DataFrame(
        np.copy(mix),
        columns=[f"mixing coeff. for the component {l}"
                 for l in np.arange(mix.shape[1])]
        )
_save_filename_extension = (f"_{_n_components}NMFcomponents_from"
                            f".csv")
_save_filename_folder = '/'.join(x for x in filename.split('/')[:-1])+'/'\
                        + filename.split('/')[-1][:-4]+'/'
if not os.path.exists(_save_filename_folder):
    os.mkdir(_save_filename_folder)

_basic_mix.to_csv(
        f"{_save_filename_folder}MixingCoeffs{_save_filename_extension}",
        sep=';', index=False)
_save_components = pd.DataFrame(
        components.T, index=sigma_kept,
        columns=[f"Component{_i}" for _i in np.arange(_n_components)])
_save_components.index.name = 'Raman shift in cm-1'
_save_components.to_csv(
        f"{_save_filename_folder}Components{_save_filename_extension}",
        sep=';')


In [41]:
pca_err = np.sum(np.abs(corrected_spectra - spectra_denoised), axis=1)
pca_err.resize(_n_y, _n_x)
plt.figure()
sns.heatmap(pca_err)#, vmin=0, vmax=1)
plt.show()